In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
mushroom=pd.read_csv('mushrooms.csv')

In [4]:
mushroom.head(5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [10]:
encoder=LabelEncoder()

In [11]:
mushroom_encode=mushroom.apply(encoder.fit_transform,axis=0)

In [12]:
mushroom_encode.head(5)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [13]:
mushroom_encode.values

array([[1, 5, 2, ..., 2, 3, 5],
       [0, 5, 2, ..., 3, 2, 1],
       [0, 0, 2, ..., 3, 2, 3],
       ...,
       [0, 2, 2, ..., 0, 1, 2],
       [1, 3, 3, ..., 7, 4, 2],
       [0, 5, 2, ..., 4, 1, 2]])

In [14]:
mushroom_encode.values.shape

(8124, 23)

In [15]:
mushroom_np=mushroom_encode.values # Turning everithing to a numpy array

In [16]:
X=mushroom_np[:,1:]

In [17]:
y=mushroom_np[:,0]

In [18]:
y.shape

(8124,)

In [19]:
X.shape

(8124, 22)

In [20]:
X

array([[5, 2, 4, ..., 2, 3, 5],
       [5, 2, 9, ..., 3, 2, 1],
       [0, 2, 8, ..., 3, 2, 3],
       ...,
       [2, 2, 4, ..., 0, 1, 2],
       [3, 3, 4, ..., 7, 4, 2],
       [5, 2, 4, ..., 4, 1, 2]])

In [21]:
y

array([1, 0, 0, ..., 0, 1, 0])

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [35]:
len(X_train), len(X_test)

(6499, 1625)

In [36]:
len(y_train), len(y_test)

(6499, 1625)

In [37]:
len(X_test)

1625

In [38]:
len(y_test)

1625

$$p(A|B)=\frac{p(B|A)p(A)}{p(B)}$$
+ P(A) = the prior, is the initial degree of belief in A.
+ P(A|B) = the posterior, is the degree of belief after incorporating news
that B is true.
+ P(B|A) = the likelihood, that can be estimated from the training data.


In [39]:
def prior(y_train,label):# p(y_train == label)=probability that y_train has label 0/1
    total=y_train.shape[0]
    true_labels=np.sum(y_train == label)
    return true_labels/total

In [40]:
y_train

array([1, 1, 1, ..., 0, 1, 0])

In [41]:
np.sum(y_train == 1)

3134

In [42]:
prior(y_train,0)

0.5177719649176796

In [43]:
def likelihood(X_train, y_train,col_num,col_val,label): # p(X|y_train == label)=p(x intersect y_train == label)/p(y_train == label)
        X_filter=X_train[y_train == label]
        num=np.sum(X_filter[:,col_num]==col_val)
        denum=X_filter.shape[0]
        
        return num/denum
    

In [44]:
likelihood(X_train,y_train,2,4,1)

0.2635609444798979

In [45]:
X_filter=X_train[y_train == 1]
X_filter[:,1]==2

array([ True,  True, False, ...,  True, False,  True])

In [87]:
def predict(X_train,y_train, x_test):
    classes=np.unique(y_train)
    number_feat=X_train.shape[1]
    post_prob=[]
    for l in classes:
        like=1.0
        for i in range(number_feat):
            cond=likelihood(X_train,y_train,i,x_test[i],l)#/(X_train.shape[0])
            like=like*cond
        pro=prior(y_train,l)
        post=like*pro
        
        post_prob.append(post)
        
        pred=np.argmax(post_prob)
    return pred

In [88]:
X_test[2]

array([5, 3, 4, 0, 2, 1, 0, 1, 0, 1, 0, 1, 2, 7, 6, 0, 2, 1, 0, 7, 4, 2])

In [89]:
X_test[2][3]

0

In [92]:
predict(X_train,y_train, X_test[4])

1

In [95]:
def accuracy(X_train,y_train,X_test,y_test):
    pred=[]
    for i in range(X_test.shape[0]):
        p=predict(X_train,y_train,X_test[i])
        pred.append(p)
        
    y_pred=np.array(pred)
    acc=np.sum(y_pred == y_test)/(y_test.shape[0])
    
    return acc

In [96]:
accuracy(X_train,y_train,X_test,y_test)

0.9963076923076923